#### k-NN on wine dataset

In [164]:
import pandas as pd
from ast import literal_eval
import re
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.decomposition import TruncatedSVD

In [165]:
def unpickle(file_path):
    # file_path = 'path/to/your/pickled_object.pkl'
    with open(file_path, 'rb') as file:
        unpickled_object = pickle.load(file)
    return unpickled_object

In [166]:
transactions = unpickle("transactions.pkl")
print(len(transactions))
print(transactions[0])

169329
['apple', 'acidity', 'sage', 'expressive', 'herb', 'tropical', 'brisk', 'citrus', 'broom', 'fruit']


In [167]:
matrix = unpickle("matrix.pkl")
print(matrix.shape)

(169329, 1060)


#### merge the binary columns with target continent

In [168]:
raw_df = pd.read_csv("03_winemag-data-reduced.csv", index_col=0)
raw_df = raw_df[['country']]
print(raw_df.shape)

(169329, 1)


/var/folders/nx/kk9254gs11j6wf22p7z4mb140000gn/T/ipykernel_28101/3048061956.py:1: DtypeWarning: Columns (9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv("03_winemag-data-reduced.csv", index_col=0)


In [169]:
import country_converter as coco
import geonamescache

# maps the country to ISO2 code 
cc = coco.CountryConverter()
iso2_codes = cc.pandas_convert(series=raw_df['country'], to='ISO2')  
raw_df['iso2'] = iso2_codes 

# get the dictionary mapping ISO2 to the continent 
gc = geonamescache.GeonamesCache()
countries = gc.get_countries()
ISO2_continent = {}
for c in countries.keys():
    ISO2_continent[c] = countries[c]['continentcode']
    
# create continent column
raw_df['continent'] = raw_df['iso2'].map(ISO2_continent)

#### train test split first

In [170]:
df = pd.DataFrame(matrix)
df['continent'] = raw_df['continent']

In [171]:
df = df.dropna()
df.shape

(169228, 1061)

In [172]:
X = df.drop(columns=['continent'])
y = df[['continent']]

In [173]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [174]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.fillna(0))
X_test_scaled = scaler.transform(X_test.fillna(0))

In [77]:
from sklearn.decomposition import PCA

def get_optimal_pca_components(matrix, min_explained_var=0.95):
    pca = PCA().fit(matrix)
    cum_sum_explained_var = np.cumsum(pca.explained_variance_ratio_)
    min_num_components = np.where(cum_sum_explained_var>=min_explained_var)[0][0] + 1
    return min_num_components

min_num_components = get_optimal_pca_components(X_train_scaled, min_explained_var=0.95)
print(min_num_components)

778


In [78]:
transformed = PCA(n_components=min_num_components).fit(X_train_scaled)

In [80]:
# apply same PCA on both X_train and X_test
# make dataframe using the PCs generated
X_train_PCA = pd.DataFrame(transformed.transform(X_train_scaled))
X_test_PCA = pd.DataFrame(transformed.transform(X_test_scaled))

In [175]:
knn = KNeighborsClassifier(n_neighbors=1000)
knn.fit(X_train_scaled, y_train)
y_pred = knn.predict(X_test_scaled)
y_pred_train = knn.predict(X_train_scaled)

print(classification_report(y_train, y_pred_train))
print("Train Accuracy:", accuracy_score(y_train, y_pred_train))
print(classification_report(y_test, y_pred))
print("Test Accuracy:", accuracy_score(y_test, y_pred))

/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

              precision    recall  f1-score   support

          AF       0.00      0.00      0.00      1782
          AS       0.00      0.00      0.00       645
          EU       0.46      0.89      0.60     61052
          NA       0.45      0.13      0.20     56931
          OC       0.00      0.00      0.00      5610
          SA       0.00      0.00      0.00      9362

    accuracy                           0.46    135382
   macro avg       0.15      0.17      0.13    135382
weighted avg       0.40      0.46      0.36    135382

Train Accuracy: 0.4559838087781241


/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          AF       0.00      0.00      0.00       443
          AS       0.00      0.00      0.00       183
          EU       0.45      0.88      0.60     15223
          NA       0.41      0.12      0.18     14225
          OC       0.00      0.00      0.00      1350
          SA       0.00      0.00      0.00      2422

    accuracy                           0.45     33846
   macro avg       0.14      0.17      0.13     33846
weighted avg       0.37      0.45      0.34     33846

Test Accuracy: 0.44516338710630504


/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [82]:
knn = KNeighborsClassifier(n_neighbors=1000)
knn.fit(X_train_PCA, y_train)

/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=1000)

In [83]:
y_pred = knn.predict(X_test_PCA)
y_pred_train = knn.predict(X_train_PCA)

print(classification_report(y_train, y_pred_train))
print("Train Accuracy:", accuracy_score(y_train, y_pred_train))
print(classification_report(y_test, y_pred))
print("Test Accuracy:", accuracy_score(y_test, y_pred))

/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

              precision    recall  f1-score   support

          AF       0.00      0.00      0.00      1782
          AS       0.00      0.00      0.00       645
          EU       0.46      0.83      0.59     61052
          NA       0.45      0.20      0.27     56931
          OC       0.00      0.00      0.00      5610
          SA       0.00      0.00      0.00      9362

    accuracy                           0.46    135382
   macro avg       0.15      0.17      0.14    135382
weighted avg       0.40      0.46      0.38    135382

Train Accuracy: 0.45738724498086897


/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          AF       0.00      0.00      0.00       443
          AS       0.00      0.00      0.00       183
          EU       0.45      0.82      0.58     15223
          NA       0.42      0.18      0.25     14225
          OC       0.00      0.00      0.00      1350
          SA       0.00      0.00      0.00      2422

    accuracy                           0.44     33846
   macro avg       0.14      0.17      0.14     33846
weighted avg       0.38      0.44      0.37     33846

Test Accuracy: 0.44347928854222063


/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### truncated SVD (instead of PCA)

In [91]:
from sklearn.decomposition import PCA

def get_optimal_svd_components(matrix, min_explained_var=0.95):
    svd = TruncatedSVD(n_components=min(matrix.shape)-1).fit(matrix)

    # Calculate cumulative explained variance
    cum_sum_explained_var = np.cumsum(svd.explained_variance_ratio_)

    # Find the minimum number of components to achieve min_explained_var
    min_num_components = np.where(cum_sum_explained_var >= min_explained_var)[0][0] + 1

    return min_num_components

min_num_components = get_optimal_svd_components(X_train_scaled, min_explained_var=0.95)
print(min_num_components)

778


In [94]:
svd = TruncatedSVD(n_components=100)
X_train_svd = svd.fit_transform(X_train.fillna(0))
X_test_svd = svd.transform(X_test.fillna(0))

knn.fit(X_train_svd, y_train)
y_pred = knn.predict(X_test_svd)
y_pred_train = knn.predict(X_train_svd)

print(classification_report(y_train, y_pred_train))
print("Train Accuracy:", accuracy_score(y_train, y_pred_train))
print(classification_report(y_test, y_pred))
print("Test Accuracy:", accuracy_score(y_test, y_pred))

/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

              precision    recall  f1-score   support

          AF       0.00      0.00      0.00      1782
          AS       0.00      0.00      0.00       645
          EU       0.46      0.81      0.58     61052
          NA       0.44      0.21      0.29     56931
          OC       0.00      0.00      0.00      5610
          SA       0.00      0.00      0.00      9362

    accuracy                           0.45    135382
   macro avg       0.15      0.17      0.15    135382
weighted avg       0.39      0.45      0.38    135382

Train Accuracy: 0.45381217591703477


/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          AF       0.00      0.00      0.00       443
          AS       0.00      0.00      0.00       183
          EU       0.45      0.79      0.57     15223
          NA       0.42      0.20      0.27     14225
          OC       0.00      0.00      0.00      1350
          SA       0.00      0.00      0.00      2422

    accuracy                           0.44     33846
   macro avg       0.14      0.17      0.14     33846
weighted avg       0.38      0.44      0.37     33846

Test Accuracy: 0.4428883767653489


/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [93]:
svd = TruncatedSVD(n_components=min_num_components)
X_train_svd = svd.fit_transform(X_train.fillna(0))
X_test_svd = svd.transform(X_test.fillna(0))

knn.fit(X_train_svd, y_train)
y_pred = knn.predict(X_test_svd)
y_pred_train = knn.predict(X_train_svd)

print(classification_report(y_train, y_pred_train))
print("Train Accuracy:", accuracy_score(y_train, y_pred_train))
print(classification_report(y_test, y_pred))
print("Test Accuracy:", accuracy_score(y_test, y_pred))

/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

              precision    recall  f1-score   support

          AF       0.00      0.00      0.00      1782
          AS       0.00      0.00      0.00       645
          EU       0.46      0.63      0.53     61052
          NA       0.44      0.40      0.42     56931
          OC       0.00      0.00      0.00      5610
          SA       0.00      0.00      0.00      9362

    accuracy                           0.45    135382
   macro avg       0.15      0.17      0.16    135382
weighted avg       0.39      0.45      0.42    135382

Train Accuracy: 0.4519507763218153


/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          AF       0.00      0.00      0.00       443
          AS       0.00      0.00      0.00       183
          EU       0.45      0.61      0.52     15223
          NA       0.42      0.38      0.40     14225
          OC       0.00      0.00      0.00      1350
          SA       0.00      0.00      0.00      2422

    accuracy                           0.44     33846
   macro avg       0.14      0.17      0.15     33846
weighted avg       0.38      0.44      0.40     33846

Test Accuracy: 0.4361519825090114


/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### try downsampling majority class

In [97]:
df.continent.value_counts(dropna=False)

continent
EU    76275
NA    71156
SA    11784
OC     6960
AF     2225
AS      828
Name: count, dtype: int64

In [98]:
# downsample EU and NA to approximately 35k each 
from sklearn.utils import resample

downsized_EU = resample(df.loc[df.continent=='EU'], replace=False, 
                        n_samples = 35000, random_state=42)
downsized_NA = resample(df.loc[df.continent=='NA'], replace=False, 
                        n_samples = 35000, random_state=42)

In [107]:
minorities = df[df.continent.isin(['SA', 'OC', 'AF', 'AS'])]

In [112]:
downsized_df = pd.concat([downsized_EU, downsized_NA, minorities], axis=0)

In [113]:
downsized_df.shape

(91797, 1061)

In [115]:
downsized_df.continent.value_counts()

continent
EU    35000
NA    35000
SA    11784
OC     6960
AF     2225
AS      828
Name: count, dtype: int64

In [116]:
X = downsized_df.drop(columns=['continent'])
y = downsized_df[['continent']]

In [117]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [118]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.fillna(0))
X_test_scaled = scaler.transform(X_test.fillna(0))
# PCA
min_num_components = get_optimal_pca_components(X_train_scaled, min_explained_var=0.95)
transformed = PCA(n_components=min_num_components).fit(X_train_scaled)

X_train_PCA = pd.DataFrame(transformed.transform(X_train_scaled))
X_test_PCA = pd.DataFrame(transformed.transform(X_test_scaled))



In [121]:
min_num_components

777

In [120]:
knn = KNeighborsClassifier(n_neighbors=1000)
knn.fit(X_train_PCA, y_train)
y_pred = knn.predict(X_test_PCA)
y_pred_train = knn.predict(X_train_PCA)

print(classification_report(y_train, y_pred_train))
print("Train Accuracy:", accuracy_score(y_train, y_pred_train))
print(classification_report(y_test, y_pred))
print("Test Accuracy:", accuracy_score(y_test, y_pred))

/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

              precision    recall  f1-score   support

          AF       0.00      0.00      0.00      1841
          AS       0.00      0.00      0.00       664
          EU       0.43      0.02      0.03     28045
          NA       0.38      0.99      0.55     27888
          OC       0.00      0.00      0.00      5551
          SA       0.00      0.00      0.00      9448

    accuracy                           0.38     73437
   macro avg       0.14      0.17      0.10     73437
weighted avg       0.31      0.38      0.22     73437

Train Accuracy: 0.3807208900145703


/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          AF       0.00      0.00      0.00       384
          AS       0.00      0.00      0.00       164
          EU       0.33      0.01      0.02      6955
          NA       0.39      0.98      0.55      7112
          OC       0.00      0.00      0.00      1409
          SA       0.00      0.00      0.00      2336

    accuracy                           0.39     18360
   macro avg       0.12      0.17      0.10     18360
weighted avg       0.28      0.39      0.22     18360

Test Accuracy: 0.3855119825708061


/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [123]:
svd = TruncatedSVD(n_components=100)
X_train_svd = svd.fit_transform(X_train.fillna(0))
X_test_svd = svd.transform(X_test.fillna(0))

knn.fit(X_train_svd, y_train)
y_pred = knn.predict(X_test_svd)
y_pred_train = knn.predict(X_train_svd)

print(classification_report(y_train, y_pred_train))
print("Train Accuracy:", accuracy_score(y_train, y_pred_train))
print(classification_report(y_test, y_pred))
print("Test Accuracy:", accuracy_score(y_test, y_pred))

/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

              precision    recall  f1-score   support

          AF       0.00      0.00      0.00      1841
          AS       0.00      0.00      0.00       664
          EU       0.39      0.53      0.45     28045
          NA       0.39      0.49      0.43     27888
          OC       0.00      0.00      0.00      5551
          SA       0.00      0.00      0.00      9448

    accuracy                           0.39     73437
   macro avg       0.13      0.17      0.15     73437
weighted avg       0.30      0.39      0.34     73437

Train Accuracy: 0.39085202282228304


/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

              precision    recall  f1-score   support

          AF       0.00      0.00      0.00       384
          AS       0.00      0.00      0.00       164
          EU       0.38      0.51      0.43      6955
          NA       0.39      0.49      0.43      7112
          OC       0.00      0.00      0.00      1409
          SA       0.00      0.00      0.00      2336

    accuracy                           0.38     18360
   macro avg       0.13      0.17      0.14     18360
weighted avg       0.29      0.38      0.33     18360

Test Accuracy: 0.38202614379084965


#### use support + downsampling

In [124]:
matrix_sup = unpickle("matrix_sup.pkl")
print(matrix_sup.shape)

(169329, 1060)


In [143]:
matrix_sup

array([[0.09527238, 0.24233554, 0.07385616, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.24233554, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.24233554, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.24233554, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.09527238, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [145]:
df = pd.DataFrame(matrix_sup)
df['continent'] = raw_df['continent']

In [146]:
downsized_EU = resample(df.loc[df.continent=='EU'], replace=False, 
                        n_samples = 12000, random_state=42)
downsized_NA = resample(df.loc[df.continent=='NA'], replace=False, 
                        n_samples = 12000, random_state=42)
minorities = df[df.continent.isin(['SA', 'OC', 'AF', 'AS'])]
downsized_df = pd.concat([downsized_EU, downsized_NA, minorities], axis=0)

In [147]:
X = downsized_df.drop(columns=['continent'])
y = downsized_df[['continent']]

In [148]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [149]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.fillna(0))
X_test_scaled = scaler.transform(X_test.fillna(0))

In [150]:
svd = TruncatedSVD(n_components=500)
X_train_svd = svd.fit_transform(X_train_scaled)
X_test_svd = svd.transform(X_test_scaled)

knn.fit(X_train_svd, y_train)
y_pred = knn.predict(X_test_svd)
y_pred_train = knn.predict(X_train_svd)

print(classification_report(y_train, y_pred_train))
print("Train Accuracy:", accuracy_score(y_train, y_pred_train))
print(classification_report(y_test, y_pred))
print("Test Accuracy:", accuracy_score(y_test, y_pred))

/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

              precision    recall  f1-score   support

          AF       0.00      0.00      0.00      1766
          AS       0.00      0.00      0.00       652
          EU       0.32      0.01      0.02      9597
          NA       0.30      0.27      0.28      9666
          OC       0.00      0.00      0.00      5567
          SA       0.27      0.79      0.40      9389

    accuracy                           0.27     36637
   macro avg       0.15      0.18      0.12     36637
weighted avg       0.23      0.27      0.18     36637

Train Accuracy: 0.2748860441630046
              precision    recall  f1-score   support

          AF       0.00      0.00      0.00       459
          AS       0.00      0.00      0.00       176
          EU       0.23      0.01      0.02      2403
          NA       0.27      0.24      0.25      2334
          OC       0.00      0.00      0.00      1393
          SA       0.27      0.77      0.40      2395

    accuracy                           0.2

/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

### try using country

In [151]:
raw_df = pd.read_csv("03_winemag-data-reduced.csv", index_col=0)
raw_df = raw_df[['country']]
df = pd.DataFrame(matrix)
df['country'] = raw_df['country']
df = df.dropna()

/var/folders/nx/kk9254gs11j6wf22p7z4mb140000gn/T/ipykernel_28101/2389161420.py:1: DtypeWarning: Columns (9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv("03_winemag-data-reduced.csv", index_col=0)


In [152]:
X = df.drop(columns=['country'])
y = df[['country']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.fillna(0))
X_test_scaled = scaler.transform(X_test.fillna(0))

min_num_components = get_optimal_pca_components(X_train_scaled, min_explained_var=0.95)
transformed = PCA(n_components=min_num_components).fit(X_train_scaled)
X_train_PCA = pd.DataFrame(transformed.transform(X_train_scaled))
X_test_PCA = pd.DataFrame(transformed.transform(X_test_scaled))

In [153]:
knn = KNeighborsClassifier(n_neighbors=1000)
knn.fit(X_train_PCA, y_train)
y_pred = knn.predict(X_test_PCA)
y_pred_train = knn.predict(X_train_PCA)

print(classification_report(y_train, y_pred_train))
print("Train Accuracy:", accuracy_score(y_train, y_pred_train))
print(classification_report(y_test, y_pred))
print("Test Accuracy:", accuracy_score(y_test, y_pred))

/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

                        precision    recall  f1-score   support

               Albania       0.00      0.00      0.00         1
             Argentina       0.00      0.00      0.00      4278
               Armenia       0.00      0.00      0.00         1
             Australia       0.00      0.00      0.00      3485
               Austria       0.00      0.00      0.00      3177
Bosnia and Herzegovina       0.00      0.00      0.00         2
                Brazil       0.00      0.00      0.00        46
              Bulgaria       0.00      0.00      0.00       112
                Canada       0.00      0.00      0.00       218
                 Chile       0.00      0.00      0.00      4909
                 China       0.00      0.00      0.00         3
               Croatia       0.00      0.00      0.00        75
                Cyprus       0.00      0.00      0.00        13
        Czech Republic       0.00      0.00      0.00        12
                 Egypt       0.00      

/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                        precision    recall  f1-score   support

             Argentina       0.00      0.00      0.00      1160
               Armenia       0.00      0.00      0.00         1
             Australia       0.00      0.00      0.00       883
               Austria       0.00      0.00      0.00       780
Bosnia and Herzegovina       0.00      0.00      0.00         1
                Brazil       0.00      0.00      0.00        11
              Bulgaria       0.00      0.00      0.00        39
                Canada       0.00      0.00      0.00        55
                 Chile       0.00      0.00      0.00      1223
               Croatia       0.00      0.00      0.00        19
                Cyprus       0.00      0.00      0.00         6
        Czech Republic       0.00      0.00      0.00         2
               England       0.00      0.00      0.00        18
                France       0.00      0.00      0.00      5424
               Georgia       0.00      

/Users/clarissajew/Desktop/da mining apriori/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [155]:
pd.Series(y_pred).value_counts()

US    33846
Name: count, dtype: int64